In [1]:
## CELL 1: LOAD ###
# run this cell to load your datafile through the GUI (currently only works with .acq files, any template)
# select the channel containing contractility and specify whether this needs an additional derivative computed

!pip install bioread
from miniMEAPsys import loadem
cont_dict,file_path=loadem()

selected contractility channel is Original Acc-dZ (0.5-20Hz)
perform highpass: True
compute derivative: False


In [2]:
## CELL 2: SET THRESHOLD AND ESTIMATE PEAKS ###
# run this cell to open a plot to decide minimum peak height
# programme will then do a rough estimate on peak values
import matplotlib.pyplot as plt
from miniMEAPsys import compute_peaks,thresh_ind

%matplotlib tk
plt.close('all')
i=thresh_ind(cont_dict['hz'])
plt.plot(cont_dict['t'][i],cont_dict['s'][i])
peak_times,peak_vals=compute_peaks(cont_dict)

plt.close('all')

In [3]:
## CELL 3: OPEN THE INTERACTIVE PLOT ###
# run this cell to open the interactive plot to visualise and amend peaks

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
from miniMEAPsys import *

%matplotlib tk
plt.close('all')
fig,ax = plt.subplots()
plt.subplots_adjust(bottom=0.25)

w = ax.plot(cont_dict['t'],cont_dict['s'])
peak_plot, = ax.plot(peak_times,peak_vals ,'vm')

# Set the axis and slider position in the plot
bottom_pos = plt.axes([0.2, 0.1, 0.65, 0.03],facecolor = 'white')
scroll_slider = Slider(bottom_pos,'time', -1,cont_dict['t'][-1])

# Make a vertically oriented slider to control the amplitude
right_pos = plt.axes([.95, 0.25, 0.0225, 0.63])
y_zoom = Slider(right_pos,label="Y zoom",valmin=0,valmax=1,valinit=.5,orientation="vertical")
left_pos = plt.axes([.05, 0.25, 0.0225, 0.63])
x_zoom = Slider(left_pos,label="X zoom",valmin=0,valmax=1,valinit=.5,orientation="vertical")

ax.set_ylim(-np.median(peak_vals)*4*.5,np.median(peak_vals)*4*.5)

ax.set_xlim(-1,4)

def update(val):
    pos = scroll_slider.val
    yzoom = y_zoom.val
    xzoom = x_zoom.val
    ax.set_xlim(pos, pos+20*xzoom)
    fig.canvas.draw_idle
    ecc=np.median(peak_vals)*4*yzoom
    ax.set_ylim(-ecc, ecc)
    fig.canvas.draw_idle
    
# update function called using on_changed() function
scroll_slider.on_changed(update)
y_zoom.on_changed(update)
x_zoom.on_changed(update)

# Display the plot
plt.show()

def onclick(event):
    global peak_times,peak_vals,fig,peak_plot
    ix, iy, ib, ik, iax = event.xdata, event.ydata, event.button, event.key, event.inaxes
    if (ix!=None) & (iax==ax):
        if (np.min(np.abs(ix-peak_times))<100) & (event.button!=1) & (ik==None):
            peak_times,peak_vals,peak_plot=remove_point(ix,peak_times,peak_vals,peak_plot)
        elif (event.button==1) & (ik==None) & (ix>0) & (ix>0) & (iy<np.max(peak_vals)*1.2): #update dist from pos
            peak_times,peak_vals,peak_plot=add_point(ix,iy,peak_times,peak_vals,peak_plot)
        elif (event.button==1)&(ik=='m'):
            peak_times,peak_vals,peak_plot=meap_dat(ix,iy,peak_times,peak_vals,peak_plot)
        fig.canvas.draw()
        fig.canvas.flush_events()
        
        if not plt.fignum_exists(1):
            canvas.mpl_disconnect(cid)
            canvas.mpl_disconnect(cidk)
            
    return peak_times,peak_vals

def onarrow(e):
    global scroll_slider
    if e.key == "right":
        pos = ax.get_xlim()
        scroll_slider.set_val(pos[0]+.500)
    elif e.key == "left":
        pos = ax.get_xlim()
        scroll_slider.set_val(pos[0]-.500)
    else:
        return

cid = fig.canvas.mpl_connect('button_press_event', onclick)
cidk = fig.canvas.mpl_connect('key_press_event', onarrow)